<a href="https://colab.research.google.com/github/ImpulsoGov/mensageria-mvp/blob/main/passo_1_mvp_algoritmo_selecao_diaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Seleção diária dos cidadãos para envio de mensagens

#### Configurações iniciais do ambiente

In [3]:
# autenticação
from google.colab import auth
auth.authenticate_user()

In [4]:
# conexão BQ
from google.cloud import bigquery
client = bigquery.Client(project='predictive-keep-314223')

In [5]:
# importação de bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from google.oauth2 import service_account
from datetime import datetime
import locale

#### Consulta e preparação dos dados

In [6]:
# histórico e divisão dos grupos de teste e controle
query = """
    SELECT *
    FROM `predictive-keep-314223.ip_mensageria_camada_prata.divisao_teste_controle_equipes`
"""

query_job = client.query(query)
df = query_job.to_dataframe()

In [7]:
# dados atualizados para confirmação de pendência
query = """
    SELECT *
    FROM `predictive-keep-314223.ip_mensageria_camada_prata.unificado_lista_com_telefones_grupos_atendimentos`
"""

query_job = client.query(query)
df_pendencias = query_job.to_dataframe()

In [8]:
# identificação dos cidadãos com pendências em cito
df_pendencias['citopatologico_pendente_atual'] = df_pendencias['status_exame'].apply(lambda x: True if x in ('exame_nunca_realizado','exame_vencido','exame_vence_no_quadrimestre_atual') else False)

In [9]:
# função para identificar crônicos em pendência
def pendencia_cronicos(x):
    if (x['esta_na_lista_hipertensos'] == True and (x['realizou_afericao_ultimos_6_meses'] == False or x['realizou_consulta_hip_ultimos_6_meses'] == False)) or (x['esta_na_lista_diabeticos'] == True and (x['realizou_consulta_dia_ultimos_6_meses'] == False or x['realizou_solicitacao_hemoglobina_ultimos_6_meses'] == False)):
        return True
    else:
        return False

df_pendencias['cronicos_pendente_atual'] = df_pendencias.apply(pendencia_cronicos,axis=1)

In [10]:
# junção dos dados históricos com a verificação de pendência atual
df_unificado = df.merge(df_pendencias[['nome_do_paciente','data_de_nascimento','municipio_id_sus','citopatologico_pendente_atual','cronicos_pendente_atual']],how='left', on=['nome_do_paciente','data_de_nascimento','municipio_id_sus'])

In [11]:
# verificaçao se a linha de cuidado ainda está pendente
def pendencia_atualizada(x):
    if x['linha_cuidado']=='cronicos' and x['cronicos_pendente_atual']==True:
        return True
    elif x['linha_cuidado']=='citopatologico' and x['citopatologico_pendente_atual']==True:
        return True
    else:
        return False

df_unificado['pendencia_atualizada'] = df_unificado.apply(pendencia_atualizada,axis=1)

In [12]:
df_unificado = df_unificado[df_unificado['pendencia_atualizada']==True]
df_unificado = df_unificado.drop_duplicates()

In [13]:
# histórico de envios anteriores
query = """
    SELECT *
    FROM `predictive-keep-314223.ip_mensageria_camada_prata.historico_envio_mensagens`
"""

query_job = client.query(query)
df_historico_envio_mensagens = query_job.to_dataframe()

In [14]:
df_unificado['chave_cidadao'] = df_unificado['nome_do_paciente'].astype(str) + '_' + df_unificado['data_de_nascimento'].astype(str)
df_historico_envio_mensagens['chave_cidadao'] = df_historico_envio_mensagens['nome_do_paciente'].astype(str) + '_' + df_historico_envio_mensagens['data_de_nascimento'].astype(str)

# filtrando cidadãos que já receberam a mensagem
df_filtrado = df_unificado[~df_unificado['chave_cidadao'].isin(df_historico_envio_mensagens['chave_cidadao'])]


In [15]:
# tratamento dos celulares

#filtrando casos com o celular preenchido incorreto
df_filtrado = df_filtrado[df_filtrado['celular_tratado']!=0]

#adicionando 55 no início do telefone
df_filtrado['celular_tratado'] = df_filtrado['celular_tratado'].astype(str)
df_filtrado['caracteres_celular'] = df_filtrado['celular_tratado'].str.len()

def trata_celular(x):
    if x['caracteres_celular']==10:
        return "55" + x['celular_tratado'][:2]+"9"+x['celular_tratado'][2:]
    elif x['caracteres_celular']==11:
        return "55" + x['celular_tratado']
    else:
        return None

df_filtrado['celular_tratado'] = df_filtrado.apply(trata_celular,axis=1)

In [16]:
# data de último exame

df_filtrado['data_exame_cito'] = df_filtrado['data_exame_cito'].astype('datetime64[ns]')
df_filtrado['data_afericao_hipertensos'] = df_filtrado['data_afericao_hipertensos'].astype('datetime64[ns]')
df_filtrado['data_exame_diabeticos'] = df_filtrado['data_exame_diabeticos'].astype('datetime64[ns]')

def ultimo_exame(x):
    if x['linha_cuidado']=='citopatologico':
        return x['data_exame_cito']
    elif x['linha_cuidado']=='cronicos' and (x['hipertensao_pendente']== True and x['diabetes_pendente']==True):
        return max(x['data_afericao_hipertensos'], x['data_exame_diabeticos'])
    elif x['linha_cuidado']=='cronicos' and x['hipertensao_pendente']== True:
        return x['data_afericao_hipertensos']
    else:
        return x['data_exame_diabeticos']
df_filtrado['data_ultimo_exame'] = df_filtrado.apply(ultimo_exame,axis=1)

In [17]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19903 entries, 0 to 30424
Data columns (total 46 columns):
 #   Column                                            Non-Null Count  Dtype         
---  ------                                            --------------  -----         
 0   municipio                                         19903 non-null  object        
 1   municipio_id_sus                                  19903 non-null  object        
 2   nome_do_paciente                                  19903 non-null  object        
 3   celular_tratado                                   19890 non-null  object        
 4   cns                                               12732 non-null  Int64         
 5   cpf                                               19903 non-null  Int64         
 6   data_de_nascimento                                19903 non-null  dbdate        
 7   ds_dim_situacao_trabalho                          19903 non-null  object        
 8   ds_identidade_genero           

#### Divisão por horários

In [18]:
# função para dividir os usuários em grupos de horários das mensagens
def dividir_grupos_equilibrado(df, num_grupos=3):
    def dividir_municipio(grupo):
        grupo_size = len(grupo)
        grupos = np.tile(range(1, num_grupos + 1), grupo_size // num_grupos + 1)[:grupo_size]
        np.random.shuffle(grupos)
        return grupos

    #considerando a divisão em equipes
    df['horario_grupo'] = df.groupby('equipe_ine')['equipe_ine'].transform(dividir_municipio)

    return df

df_dividido = dividir_grupos_equilibrado(df_filtrado)

In [19]:
# máximo de 15 pessoas por equipe, dia e linha de cuidado -> máximo de 5 pessoas por horário, equipe, dia e linha de cuidado
df_envio_diario = df_dividido.groupby(['municipio','equipe_ine','linha_cuidado','horario_grupo','grupo']).apply(lambda x: x.sample(min(len(x), 5))).reset_index(drop=True)

<ipython-input-19-1753165cde40>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_envio_diario = df_dividido.groupby(['municipio','equipe_ine','linha_cuidado','horario_grupo','grupo']).apply(lambda x: x.sample(min(len(x), 5))).reset_index(drop=True)


In [20]:
# ajuste no formato da coluna de tipo de grupo
dia_semana = datetime.now().strftime('%a').upper()
df_envio_diario['mvp_tipo_grupo'] = dia_semana+'_H'+df_envio_diario['horario_grupo'].astype(str).str.zfill(2)

In [21]:
df_envio_diario

,municipio,municipio_id_sus,nome_do_paciente,celular_tratado,cns,cpf,data_de_nascimento,ds_dim_situacao_trabalho,ds_identidade_genero,no_social_cidadao,...,linha_cuidado,grupo,citopatologico_pendente_atual,cronicos_pendente_atual,pendencia_atualizada,chave_cidadao,caracteres_celular,data_ultimo_exame,horario_grupo,mvp_tipo_grupo
0,Alagoinha,260060,ANA LARISSA SILVA ALVES DE OLIVEIRA,5587981353676,203388596880008,13263953435,1998-09-03,Outro,Não informado,None,...,citopatologico,controle,True,False,True,ANA LARISSA SILVA ALVES DE OLIVEIRA_1998-09-03,11,NaT,1,FRI_H01
1,Alagoinha,260060,ALVANI LOPES OLIVEIRA,5587981192986,708003350123929,79915396449,1971-06-28,Outro,Não informado,None,...,citopatologico,controle,True,True,True,ALVANI LOPES OLIVEIRA_1971-06-28,11,2020-11-09,1,FRI_H01
2,Alagoinha,260060,AMANDA COSTA DE SIQUEIRA,5597981455629,<NA>,12065143452,1993-11-18,Outro,Não informado,None,...,citopatologico,controle,True,False,True,AMANDA COSTA DE SIQUEIRA_1993-11-18,11,NaT,1,FRI_H01
3,Alagoinha,260060,MARIA JOCILENE ALMEIDA DE ANDRADE,5587981538392,<NA>,71244010464,1997-08-02,Outro,Não informado,None,...,citopatologico,controle,True,False,True,MARIA JOCILENE ALMEIDA DE ANDRADE_1997-08-02,10,NaT,1,FRI_H01
4,Alagoinha,260060,RAFAELA ARAUJO GALINDO,5587981769413,<NA>,11481638408,1996-04-22,Outro,Homem transgênero,None,...,citopatologico,controle,True,False,True,RAFAELA ARAUJO GALINDO_1996-04-22,11,2021-10-18,1,FRI_H01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,Vitorino Freire,211300,MARIA DA CONCEIÇÃO VIEIRA BRITO,5598984180893,704208715010789,72098341334,1938-09-01,Aposentado / Pensionista,Não informado,None,...,cronicos,teste,False,True,True,MARIA DA CONCEIÇÃO VIEIRA BRITO_1938-09-01,11,2024-06-14,3,FRI_H03
3146,Vitorino Freire,211300,ANTONIO BRAGA SILVA,5598984385792,702609202280342,78360366268,1975-08-13,Não informado,Não informado,None,...,cronicos,teste,False,True,True,ANTONIO BRAGA SILVA_1975-08-13,11,NaT,3,FRI_H03
3147,Vitorino Freire,211300,MANOEL DA PAZ MENESES,5598984106610,700705480296580,80492975353,1948-01-24,Não informado,Não informado,None,...,cronicos,teste,False,True,True,MANOEL DA PAZ MENESES_1948-01-24,10,2024-06-04,3,FRI_H03
3148,Vitorino Freire,211300,MARIA CREUSA SILVA DA CONCEIÇAO,5598936552747,706205594939166,49326163334,1948-11-15,Aposentado / Pensionista,Não informado,None,...,cronicos,teste,False,True,True,MARIA CREUSA SILVA DA CONCEIÇAO_1948-11-15,10,2024-06-12,3,FRI_H03


In [22]:
df_envio_diario= df_envio_diario.rename(columns={'grupo':'mvp_grupo'})
df_envio_diario['data_ultimo_exame_cito'] = df_envio_diario['data_exame_cito'].astype('datetime64[ns]')
df_envio_diario['data_ultima_afericao_hipertensos'] = df_envio_diario['data_afericao_hipertensos'].astype('datetime64[ns]')
df_envio_diario['data_ultimo_exame_diabeticos'] = df_envio_diario['data_exame_diabeticos'].astype('datetime64[ns]')
df_envio_dia_atual = df_envio_diario[['municipio','municipio_id_sus', 'equipe_ine', 'equipe_nome', 'linha_cuidado','nome_do_paciente','data_de_nascimento','celular_tratado','mvp_tipo_grupo','mvp_grupo','numero_visitas_ubs_ultimos_12_meses','data_ultimo_exame_cito','data_ultima_afericao_hipertensos','data_ultimo_exame_diabeticos']]
df_envio_dia_atual['mvp_data_envio'] = datetime.today().strftime('%Y-%m-%d')
df_envio_dia_atual['mvp_data_envio'] = df_envio_dia_atual['mvp_data_envio'].astype('datetime64[ns]')
df_envio_dia_atual['celular_tratado'] = df_envio_dia_atual['celular_tratado'].astype(str)
df_envio_dia_atual['mvp_tipo_grupo'] = df_envio_dia_atual['mvp_tipo_grupo'].astype(str)
df_envio_dia_atual['mvp_data_envio'] = df_envio_dia_atual['mvp_data_envio'].astype('datetime64[ns]')
df_envio_dia_atual['mvp_grupo'] = df_envio_dia_atual['mvp_grupo'].astype('str')
df_envio_dia_atual['numero_visitas_ubs_ultimos_12_meses'] = df_envio_dia_atual['numero_visitas_ubs_ultimos_12_meses'].astype(int)

<ipython-input-22-e7c763582cb5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_envio_dia_atual['mvp_data_envio'] = datetime.today().strftime('%Y-%m-%d')
<ipython-input-22-e7c763582cb5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_envio_dia_atual['mvp_data_envio'] = df_envio_dia_atual['mvp_data_envio'].astype('datetime64[ns]')
<ipython-input-22-e7c763582cb5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [23]:
#adicionar dados na tabela de histórico
table_id = "predictive-keep-314223.ip_mensageria_camada_prata.historico_envio_mensagens"
# incremento com os dados do dia atual
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
job = client.load_table_from_dataframe(df_envio_dia_atual, table_id, job_config=job_config)
job.result()

LoadJob<project=predictive-keep-314223, location=US, id=e02ee37a-fc00-4d6b-8a76-2a2ac03dc215>